# Characteristics Exploration

Imports

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt
import matplotlib
import tqdm.auto as tqdm

import os
from PIL import Image
from sklearn.metrics import accuracy_score
import torchvision
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import torchvision.models as models

plt.style.use('seaborn')

import DiagnosisFunctions.tools as tools

import albumentations as A
import torchvision.transforms.functional as TF

Get splits

In [2]:
(train_path, train_target), (val_path, val_target), (test_path, test_target), le = tools.get_splits_characteristics()

train_set    = tools.CharacteristicsDataset(path = train_path, target = train_target, size = [200, 200])
train_loader = DataLoader(train_set, batch_size=16)

val_set    = tools.CharacteristicsDataset(path = val_path, target = val_target, size = [200, 200])
val_loader = DataLoader(val_set, batch_size=16)

test_set    = tools.CharacteristicsDataset(path = test_path, target = test_target, size = [200, 200])
test_loader = DataLoader(test_set, batch_size=16)

# Create network

In [3]:
n_classes = 8
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        resnet = models.resnext50_32x4d(pretrained=True)
        resnet.fc = nn.Sequential(
            #nn.Dropout(p=0.2),
            nn.Linear(in_features=resnet.fc.in_features, out_features=n_classes)
        )
        self.base_model = resnet
        self.sigm = nn.Sigmoid()

    def forward(self, x):
        return self.sigm(self.base_model(x))

In [12]:
cnn = CNN()

criterion = nn.BCELoss()
optimizer = optim.Adam(cnn.parameters())

num_epochs = 10

train_losses     = []
train_accuracies = []

val_losses       = []
val_accuracies   = []


for epoch in tqdm.tqdm(range(num_epochs), unit='epoch', desc="Epoch"):

    #Epochs start @ 1 now.
    epoch += 1

    ## -- Training -- ##
    cnn.train()
    train_loss = 0
    predictions  = []
    ground_truth = []
    
    for data in train_loader:

        #Fetch images and targets from train loader.
        images, targets = data
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = cnn(images)
        
        loss = criterion(outputs, torch.FloatTensor(targets))
        loss.backward()
        optimizer.step()

        #Add the batch loss
        train_loss += loss.item()

        #Save predictions and targets
        predictions  += outputs.tolist()
        ground_truth += targets.tolist()
        
    #Append this epoch's statistics.
    train_losses.append(train_loss)
    train_accuracies.append(accuracy_score(ground_truth, predictions))
    
    ## -- End of Training -- ##
    
    ## -- Validation -- ##
    cnn.eval()
    val_loss = 0
    predictions  = []
    ground_truth = []
    
    for data in val_loader:
    
        #Fetch images and targets from val loader.
        images, targets = data
        
        #Get that stuff on the GPU
        #images  = images.to(device)
        #targets = targets.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = cnn(images)
        
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        #Add the batch loss
        val_loss += loss.item()

        #Save predictions and targets
        predictions  += outputs.tolist()
        ground_truth += targets.tolist()
        
    #Append this epoch's statistics.
    val_losses.append(val_loss)
    val_accuracies.append(accuracy_score(ground_truth, predictions))
    ## -- End of Validation -- ##

Epoch:   0%|          | 0/10 [00:00<?, ?epoch/s]

ValueError: only one element tensors can be converted to Python scalars